# CS838 Project Stage3

Yuying Chen
Maria Kamenetsky

In [1]:
#!which -a python
#!echo $PATH

/cygdrive/c/Anaconda3/python
/usr/bin/python
/cygdrive/c/Anaconda3/python
/cygdrive/c/users/maria/appdata/local/enthought/canopy/user/scripts/python
/cygdrive/c/Users/Maria/AppData/Local/Enthought/Canopy/User/python
/cygdrive/c/Users/Maria/AppData/Local/Enthought/Canopy/User/Scripts/python


In [4]:
#import modules
import csv
import py_entitymatching as em
import os
import sys
import re
import time
import shutil
import collections
import pandas as pd
from sklearn import tree
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import scipy
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.model_selection import KFold
import random

## Read input tables

Since the original two movie dataset that we selected don't have too many overlaps, we changed one of them to another movie dataset. The first dataset is from kaggle.com. The second dataset is from movielens.org.

We added an id column for both datasets.

In [11]:
A = em.read_csv_metadata('../../DataforProject/movie1_stage3.csv', key='id',encoding='mac_roman')
B = em.read_csv_metadata('../../DataforProject/movie2_stage3.csv', key='movieId',encoding='mac_roman')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [12]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A x B: ' + str(len(A)*len(B)))

Number of tuples in A: 5043
Number of tuples in B: 27278
Number of tuples in A x B: 137562954


In [7]:
A.head()

,id,movie_title,genres,director_name,plot_keywords,actor_3_facebook_likes,cast_total_facebook_likes,content_rating,budget,duration,...,actor_2_facebook_likes,imdb_score,movie_facebook_likes,language,actor_1_facebook_likes,facenumber_in_poster,actor_1_name,num_user_for_reviews,aspect_ratio,num_critic_for_reviews
0,1,avatar,action|adventure|fantasy|sci-fi,james cameron,avatar|future|marine|native|paraplegic,855,4834,pg-13,237000000,178,...,936,7.9,33000,english,1000,0,cch pounder,3054,1.78,723
1,2,pirates of the caribbean: at world's end,action|adventure|fantasy,gore verbinski,goddess|marriage ceremony|marriage proposal|pirate|singapore,1000,48350,pg-13,300000000,169,...,5000,7.1,0,english,40000,0,johnny depp,1238,2.35,302
2,3,spectre,action|adventure|thriller,sam mendes,bomb|espionage|sequel|spy|terrorist,161,11700,pg-13,245000000,148,...,393,6.8,85000,english,11000,1,christoph waltz,994,2.35,602
3,4,the dark knight rises,action|thriller,christopher nolan,deception|imprisonment|lawlessness|police officer|terrorist plot,23000,106759,pg-13,250000000,164,...,23000,8.5,164000,english,27000,0,tom hardy,2701,2.35,813
4,5,star wars: episode vii - the force awakens,documentary,doug walker,.,.,143,.,.,.,...,12,7.1,0,.,131,0,doug walker,.,.,.


In [13]:
B.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [15]:
em.get_key(A), em.get_key(B)

('id', 'movieId')

Since the tables are very large let's downsample:

In [17]:
A1, B1 = em.down_sample(A,B, 200,1, show_progress=True)
len(A1), len(B1)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


(170, 200)

## Block Tables To Get Candidate Set

Block on attribute 'year' using an overlap blocker since the same movie must have the same year. The output attributes of table A is limited to id, title, year and genres to match the information provided by table B.

In [18]:
# Create overlap blocker
ob = em.OverlapBlocker()
# Block tables using 'year' attribute 
#C = ob.block_tables(sample_A, sample_B, 'title_year', 'year', 
 #                   l_output_attrs=['id', 'movie_title', 'title_year', 'genres'], 
#                  r_output_attrs=['movieId', 'title', 'year', 'genres'],
   #                 overlap_size=1, show_progress=True
                    #)
C = ob.block_tables(A, B, 'title_year', 'year',
                   l_output_attrs=['id', 'movie_title', 'title_year', 'genres'],
                   r_output_attrs=['movieId', 'title','year', 'genres'],
                   overlap_size=1, show_progress=True)
em.to_csv_metadata(C, './candidate_pairs.csv') #I really like this function

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:16


True

In [19]:
len(C)

3288743

## Match tuple pairs in candidate set

### Import Pre-Determined Golden Data

We randomly sampled 500 tuple pairs for labeling. There are 201 matches and 299 mismatches. **gold.csv** is our gold dataset of perfect matches that we have determined prior to this. 

In [24]:
G = pd.read_csv("../../DataforProject/gold.csv", encoding='mac_roman')
len(G)

500

In [25]:
#G = em.read_csv_metadata('gold.csv', 
 #                        key='_id',
    #                        ltable=A, rtable=B, 
#                        fk_ltable='ltable_id', fk_rtable='rtable_movieId',
 #                        encoding='mac_roman')
#len(G)

In [26]:
G.head(10)

,_id,ltable_id,rtable_movieId,ltable_movie_title,ltable_title_year,ltable_genres,rtable_title,rtable_year,rtable_genres,label
0,2433,894,64716,seven pounds,2008,drama|romance,Seven Pounds,2008,Drama,1
1,4545,2842,4015,"dude, where's my car?",2000,comedy|mystery,"Dude, Where's My Car?",2000,Comedy|Sci-Fi,1
2,6907,3697,104908,insidious: chapter 2,2013,fantasy|horror|thriller,Insidious: Chapter 2,2013,Horror|Thriller,1
3,24168,4384,2891,"happy, texas",1999,comedy|crime|romance,"Happy, Texas",1999,Comedy,1
4,31352,1844,7980,a bridge too far,1977,drama|history|war,"Bridge Too Far, A",1977,Action|Drama|War,1
5,35987,874,2676,instinct,1999,drama|thriller,Instinct,1999,Drama|Thriller,1
6,37526,2582,1599,steel,1997,action|crime|sci-fi,Steel,1997,Action,1
7,50124,1212,1408,the last of the mohicans,1992,action|adventure|drama|romance|war,"Last of the Mohicans, The",1992,Action|Romance|War|Western,1
8,57296,2987,66200,two lovers,2008,drama|romance,Two Lovers,2008,Drama|Romance,1
9,64913,2952,2841,stir of echoes,1999,horror|mystery|thriller,Stir of Echoes,1999,Horror|Mystery|Thriller,1


### Splitting Labeled Data into Development and Evaluation Set